# Chat completions

In [1]:
base_url = "API URL" # e.g. "http://localhost:8000/v1/"
api_key = "YOUR API KEY"

## Without chat history

In [3]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [4]:
# unstreamed chat
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Salut Albert !"}],
    "stream": False,
    "n": 1,
}

client.chat.completions.create(**data)

ChatCompletion(id='cmpl-2fa6f08162cb43b8bd8b38920090a033', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Bonjour ! Je suis Albert, un modèle linguistique d'intelligence artificielle. Je suis ravi de discuter avec toi ! Qu'est-ce que tu veux parler ou demander ?", role='assistant', function_call=None, tool_calls=[]), stop_reason=128009)], created=1720595202, model='AgentPublic/llama3-instruct-8b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=41, prompt_tokens=14, total_tokens=55))

## With chat history

Because chat history need a `id` parameters (for chat ID) and this parameters is not currently supported by official OpenAI Python client, you need to request Albert API by *requests* npackage (native Python). We currently works to deploy a Albert Python client to support this parameters.   

In [5]:
# Request configuration
import requests

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

The chat will keep the history of the conversation as long as the `user` parameter is set. 

In [19]:
# unstreamed chat
user = "leo"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content":"Salut Albert !"}],
    "stream": False,
    "n": 1,
    "user": user, # to start a conversation with chat history, setup a user name
}

response = session.post(url=f"{base_url}/chat/completions", json=data)

response.json()

{'id': '91d4ed10-9ba4-409c-8c15-01548f7f8a68',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "Salut ! Comment vas-tu aujourd'hui ? Je suis Albert, un modèle de langage artificiel, et je suis là pour discuter avec toi ! Qu'est-ce que tu veux parler de ?",
    'role': 'assistant',
    'function_call': None,
    'tool_calls': []},
   'stop_reason': 128009}],
 'created': 1720595309,
 'model': 'AgentPublic/llama3-instruct-8b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 44, 'prompt_tokens': 14, 'total_tokens': 58}}

Now, to continue the conversation with the current chat history, you need to pass the `id` parameter that was returned in the previous template response.

In [18]:
chat_id = response.json()["id"]

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content":"Quel est mon précédent message ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "id": chat_id # to follow a conversation with chat history, setup the previous chat id
}
response = session.post(url=f"{base_url}/chat/completions", json=data)

response.json()

{'id': 'd433a128-8b53-42bd-b89f-156c75e102b3',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Votre précédent message est également : "Salut Albert !"',
    'role': 'assistant',
    'function_call': None,
    'tool_calls': []},
   'stop_reason': 128009}],
 'created': 1720595300,
 'model': 'AgentPublic/llama3-instruct-8b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 14, 'prompt_tokens': 162, 'total_tokens': 176}}

You can resume a past conversation by passing the `id` parameter and the correct chat ID. 